In [11]:
#import Pkg; Pkg.add("Pkg")
#import Pkg; Pkg.add("LightGraphs")
#import Pkg; Pkg.add("PyCall")
#import Pkg; Pkg.add("MatrixNetworks")
#import Pkg; Pkg.add("SparseArrays")
#import Pkg; Pkg.add("LinearAlgebra")
#import Pkg; Pkg.add("VegaLite")
#import Pkg; Pkg.add("MultivariateStats")
#import Pkg; Pkg.add("RDatasets")
#import Pkg; Pkg.add("TensorOperations")
#import Pkg; Pkg.add("Colors")
#import Pkg; Pkg.add("GraphPlot")
#import Pkg; Pkg.add("SimpleWeightedGraphs")
#import Pkg; Pkg.add("DelimitedFiles")
#import Pkg; Pkg.add(" StatsBase")
#import Pkg; Pkg.add("Plots")
#import Pkg; Pkg.add("VegaDatasets")
#import Pkg; Pkg.add("TSne")



using PyCall
using LightGraphs
using MatrixNetworks
using VegaDatasets
using DataFrames
using SparseArrays
using LinearAlgebra
using Plots
using VegaLite
using TensorOperations
using TSne, Colors, GraphPlot, LightGraphs, SimpleWeightedGraphs,DelimitedFiles, StatsBase, Node2Vec

In [12]:
function uniform(size, tensor)
        if tensor is 
    not
    nothing
        
    return 
    bound = 1.0 / math.sqrt(size)
    tensor.data.uniform_(-bound, bound) 
       
    end

end


uniform (generic function with 1 method)

In [13]:
function kaiming_uniform(tensor, fan, a)
    if tensor is 
        not 
        nothing
        
        return
        bound = math.sqrt(6 / ((1 + a^2) * fan))
        tensor.data.uniform_(-bound, bound)
    end
end

function glorot(tensor)
    if tensor is 
        not 
        nothing
        
        return
        stdv = math.sqrt(6.0 / (tensor.size(-2) + tensor.size(-1)))
        tensor.data.uniform_(-stdv, stdv)
    end
end


function glorot_orthogonal(tensor, scale)
    if tensor is 
        not 
        nothing
        
        return
        torch.nn.init.orthogonal_(tensor.data)
        scale /= ((tensor.size(-2) + tensor.size(-1)) * tensor.var())
        tensor.data *= scale.sqrt()
    end
end

function zeros(tensor)
    if tensor is 
        not
        nothing
        
        return
        tensor.data.fill_(0)
    end
end


function ones(tensor)
    if tensor is 
        not
        nothing
        
        return
        tensor.data.fill_(1)
    end
end


function normal(tensor, mean, std)
    if tensor is 
        not 
        nothing
        
        return
        tensor.data.normal_(mean, std)
    end
end




normal (generic function with 1 method)

In [14]:
hasproperty(item, s::Symbol) = s in fieldnames(typeof(item))

hasproperty (generic function with 1 method)

In [15]:
#'r' = 'reset_parameters'
function _reset(item)
    if hasproperty(item, 'r') 
        return
        item.reset_parameters()
    end
end

_reset (generic function with 1 method)

In [16]:
#'c'='children'
function reset(nn)
    if nn is
        not
        nothing
                
        return
        if hasproperty(nn, 'c') & len(list(nn.children())) > 0
        end
        for item in nn.children()
            return _reset(item)
            end

        if not
                    hasproperty(nn, 'c') & len(list(nn.children())) > 0
                    return
                    _reset(nn) 
        end
                    
                    function _reset(item)
                        if hasproperty(item, 'r') 
                        return
                        item.reset_parameters()
                    end
        end
    end
end

                    

reset (generic function with 1 method)

In [17]:
using TensorOperations

In [18]:
#using SparseArray, TensorOperations, Linearalgerbra
#from torch.nn import Module, Parameter#
#from torch import Tensor
#from typing import Optional
#using glorot

function gcn_norm(SparseArray, add_self_loops; bool = True)
    if adj.has_value() == False
        adj = adj.fill_value(1.)
    if add_self_loops
        A_tilde = fill_diag(adj, 1)
    D_tilde = sum(A_tilde, dim = 1)
    D_tilde_inv_sqrt = D_tilde.pow_(-0.5)
    D_tilde_inv_sqrt.masked_fill_(D_tilde_inv_sqrt == float('i'), 0)
    A_hat = mul(mul(A_tilde, D_tilde_inv_sqrt.unsqueeze(0)), D_tilde_inv_sqrt.unsqueeze(0))
    return A_hat
        end

methods(GCN_layer(Module))
    
    function __init__(self, 
                 in_channels, 
                 out_channels, 
                 add_self_loops; bool = True,
                 weight_init = glorot
                )
        super(GCN_layer, self).__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.add_self_loops = add_self_loops
        self.weights = Parameter(data = Tensor(in_channels, out_channels))
        weight_init(self.weights)
        end
end    
    function forward(self, X, edges)
        A_hat = gcn_norm(adj = edges, 
                         add_self_loops = self.add_self_loops
                        )
        out = matmul(A_hat, X) 
        
        return out
    end
end

gcn_norm (generic function with 1 method)

In [19]:
] add https://github.com/ollin18/Node2Vec.jl

    Updating git-repo `https://github.com/ollin18/Node2Vec.jl`
   Resolving package versions...
  No Changes to `C:\Users\Toshiba\.julia\environments\v1.6\Project.toml`
  No Changes to `C:\Users\Toshiba\.julia\environments\v1.6\Manifest.toml`


In [45]:
eps = 1e-15

#struct(node2vec_layer(Module))
#end
#methods(node2vec_layer(Module))

    function __init__(self, graph, node_type)
        super(node2vec_layer, self).__init__()
        self.w = graph.node_data[node_type]
end

    
    function forward(self, batch)
        """Returns the embeddings for the nodes in :obj:`batch`."""
        return embedding(batch, self.w, sparse = True)
end
    
    function loss(self, pos_rw, neg_rw)
        r"""Computes the loss given positive and negative random walks."""

        # Positive loss.
        start, rest = pos_rw[:, 0], pos_rw[:,:].contiguous()

        h_start = embedding(start, self.w, sparse = True).view(pos_rw.size(0), 1,
                                                          self.w.shape[1])
        h_rest = embedding(rest.view(-1), self.w, sparse = True).view(pos_rw.size(0), -1,
                                                                 self.w.shape[1])

        out = (h_start * h_rest).sum(dim=-1).view(-1)
        pos_loss = -log(sigmoid(out) + eps).mean()

        # Negative loss.
        start, rest = neg_rw[:, 0], neg_rw[:, :].contiguous()

        h_start = embedding(start, self.w, sparse = True).view(neg_rw.size(0), 1,
                                                          self.w.shape[1])
        h_rest = embedding(rest.view(-1), self.w, sparse = True).view(neg_rw.size(0), -1,
                                                                 self.w.shape[1])

        out = (h_start * h_rest).sum(dim=-1).view(-1)
        neg_loss = -log(1 - sigmoid(out) + eps).mean()

        return pos_loss + neg_loss
    end
    
    


loss (generic function with 1 method)